# Kaggle link: https://www.kaggle.com/code/l1ghtsource/yappy-hackathon

# Данный ноутбук содержит реализации всех необходимых для работы классов и функций, а также замеры времени работы на CPU и GPU (Tesla P100-PCIE-16GB)

# Доп. информация о CPU:

Architecture:          x86_64 \
CPU op-mode(s):        32-bit, 64-bit \
Byte Order:            Little Endian \
CPU(s):                4 \
On-line CPU(s) list:   0-3 \
Thread(s) per core:    2 \
Core(s) per socket:    2 \
Socket(s):             1 \
NUMA node(s):          1 \
Vendor ID:             GenuineIntel \
CPU family:            6 \
Model:                 79 \
Model name:            Intel(R) Xeon(R) CPU @ 2.20GHz \
Stepping:              0 \
CPU MHz:               2199.998 \
BogoMIPS:              4399.99 \
Hypervisor vendor:     KVM \
Virtualization type:   full \
L1d cache:             32K \
L1i cache:             32K \
L2 cache:              256K \
L3 cache:              56320K \
NUMA node0 CPU(s):     0-3

In [1]:
import os
import glob

def find_mp4_files(directory):
    pattern = os.path.join(directory, '*.mp4')
    mp4_files = glob.glob(pattern)
    
    return mp4_files

# directory_path = '/kaggle/input/hack-chunk-2/2' # со старого хака для теста
# mp4_files = find_mp4_files(directory_path)

In [2]:
from IPython.display import HTML
from base64 import b64encode

def play(filename): # плеер для видео
    html = ''
    video = open(filename,'rb').read()
    src = 'data:video/mp4;base64,' + b64encode(video).decode()
    html += '<video width=1000 controls autoplay loop><source src="%s" type="video/mp4"></video>' % src
    return HTML(html)

# play(mp4_files[20])

In [3]:
! pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 44.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 4.4 MB/s eta 0:00:00


In [4]:
import subprocess
import sys
from pathlib import Path

import click
import pandas as pd
import torch
import torchaudio
from torchaudio.transforms import Resample
from tqdm import tqdm
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor

2024-06-11 10:45:14.941131: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-11 10:45:14.941298: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-11 10:45:15.102815: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
working_dir = '/kaggle/working'

if not os.path.exists(os.path.join(working_dir, 'tmp')):
    os.makedirs(os.path.join(working_dir, 'tmp'))

In [6]:
! pip install deep_translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.4 MB/s eta 0:00:00


In [7]:
from deep_translator import GoogleTranslator

# Затюненный переводчик на замену GoogleTranslator

#### TRANSLATOR
#### w/ google -> 94ms
#### w/ transformers -> 103ms on GPU and 262ms on CPU, but (!) it's free for commercial use 

In [8]:
import torch
from transformers import MarianMTModel, MarianTokenizer


class MarianTranslator:
    def __init__(self):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.model = MarianMTModel.from_pretrained('lightsource/yappy-fine-tuned-opus-mt-ru-en').to(self.device)
        self.tokenizer = MarianTokenizer.from_pretrained('lightsource/yappy-fine-tuned-opus-mt-ru-en')

    def translate(self, text):
        inputs = self.tokenizer(text, return_tensors='pt', padding=True, truncation=True).to(self.device)
        translated = self.model.generate(**inputs)

        translated_text = [self.tokenizer.decode(t, skip_special_tokens=True, truncation=True) for t in translated]

        return translated_text[0]

# ASR модель + VAD

#### ASR (test on 25sec video)
#### Whisper-medium: 2.68s on GPU, ~30s on CPU (USE THIS IF U HAVE GPU)
#### Whisper-base: 6s on CPU (USE THIS IF U HAVE ONLY CPU)
#### Whisper-tiny: 3.5s on CPU (ALSO U CAN USE THIS FOR CPU INFERENCE)

In [9]:
class WhisperModel():
    def __init__(self):
        flag = torch.cuda.is_available()
        
        self.model_id = 'openai/whisper-medium' if flag else 'openai/whisper-base'
        self.dtype = torch.float16 if flag else torch.float32
        self.device = 'cuda' if flag else 'cpu'

        model = AutoModelForSpeechSeq2Seq.from_pretrained(
            self.model_id, torch_dtype=self.dtype, low_cpu_mem_usage=True, use_safetensors=True
        ).to(self.device)

        processor = AutoProcessor.from_pretrained(self.model_id)
        
        self.model = model
        self.processor = processor
        self.forced_decoder_ids_eng = processor.get_decoder_prompt_ids(language='english', task='transcribe')
        self.resampler = {}
        self.vad, self.vad_utils = torch.hub.load(repo_or_dir='snakers4/silero-vad',
                                                  model='silero_vad',
                                                  force_reload=True,
                                                  onnx=True)

    def process_sample(self, filename):
        print(filename)
        
        try:
            code = subprocess.call(
                [
                    'ffmpeg',
                    '-y',
                    '-i',
                    filename,
                    '-vn',
                    '/tmp/audio.wav'
                ], 
                stderr=subprocess.DEVNULL, 
                stdout=subprocess.DEVNULL
            )
            
            if code != 0:
                raise ValueError()
            
            clip, clip_hz = torchaudio.load('/tmp/audio.wav', backend='ffmpeg')
            
        except:
            return ''
        
        if clip_hz != 16000:
            
            if clip_hz not in self.resampler:
                self.resampler[clip_hz] = Resample(clip_hz, 16000)
            clip = self.resampler[clip_hz](clip)
            
        clip = clip.mean(dim=0)  # stereo to mono
        speech_ts = self.vad_utils[0](clip.unsqueeze(0), self.vad, sampling_rate=16000, threshold=0.1)
        
        full = []
        for ts in speech_ts:
            
            with torch.inference_mode():
                input_features = self.processor(
                    clip[ts['start']:ts['end'] + 1], sampling_rate=16000, return_tensors='pt'
                ).input_features.to(self.dtype).to(self.device)
                
                predicted_ids_eng = self.model.generate(input_features, 
                                                        forced_decoder_ids=self.forced_decoder_ids_eng, 
                                                        do_sample=True,
                                                        temperature=0.3)
                transcription_eng = self.processor.batch_decode(predicted_ids_eng, skip_special_tokens=True)
            
                full.append(transcription_eng[0])

        return ' '.join(full)

In [19]:
whisper = WhisperModel()

config.json:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.06G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.75k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/conda/lib/python3.10/site-packages/torch/hub.py:294: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/snakers4/silero-vad/zipball/master" to /root/.cache/torch/hub/master.zip


In [32]:
%%time 
whisper.process_sample('/kaggle/input/yappy-videos/fhd (1).mp4') # testing on cpu (base model)

/kaggle/input/yappy-videos/fhd (1).mp4
CPU times: user 10.8 s, sys: 47.2 ms, total: 10.9 s
Wall time: 5.91 s


" what is different from the Japanese government, from the Japanese government to other countries. When in Japan there are some kind of ecotachism, for example, well, as nature is, then the head of this prefecture is with its family, with its wife, leaving the region and being settled in the country, and being settled with all the countries that are suffering, while choosing the best conditions. And they live like that until they destroy the entire situation that is complex. And now let's remember the head of their region."

In [21]:
%%time 
whisper.process_sample('/kaggle/input/yappy-videos/fhd (1).mp4') # testing on gpu (medium model)

/kaggle/input/yappy-videos/fhd (1).mp4
CPU times: user 2.57 s, sys: 44.1 ms, total: 2.62 s
Wall time: 2.68 s


' What is the difference between Japanese government and government from other countries? When some cataclysms occur in Japan, for example, well, as a rule, natural, then the head of this prefecture, together with his family, together with his wife, leaves for this region and settles along with all the victims, while choosing the worst conditions. And they live like this until they manage the entire situation. But now, remember the head of your region.'

In [10]:
! pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 3.4 MB/s eta 0:00:00 0:00:01


In [1]:
! pip install av

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 42.9 MB/s eta 0:00:00:00:0100:01


In [2]:
import easyocr
import cv2
import av
import numpy as np

from PIL import Image

from transformers import AutoProcessor, AutoModel
from transformers import CLIPVisionModelWithProjection
from transformers import CLIPTokenizer, CLIPTextModelWithProjection
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize, InterpolationMode

2024-06-11 11:40:13.782806: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-11 11:40:13.782904: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-11 11:40:13.906946: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# CLIP для извлечения фичей из видеосоставляющей

#### CLIP (test on 20 sec video)
#### 4.3s on GPU and 7.5s on CPU

In [4]:
class CLIPmodel:
    def __init__(self):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model = CLIPVisionModelWithProjection.from_pretrained("Searchium-ai/clip4clip-webvid150k").to(self.device)
        self.processor = AutoProcessor.from_pretrained("Searchium-ai/clip4clip-webvid150k")
        
    def video2image(self, video_path, frame_rate=1.0, size=224):
        def preprocess(size, n_px):
            return Compose([
                Resize(size, interpolation=InterpolationMode.BICUBIC),            
                CenterCrop(size),
                lambda image: image.convert("RGB"),
                ToTensor(),
                Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711)),
            ])(n_px)

        cap = cv2.VideoCapture(video_path, cv2.CAP_FFMPEG)
        frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        
        if fps < 1:
            images = np.zeros([3, size, size], dtype=np.float32) 
            print("ERROR: problem reading video file: ", video_path)
        else:
            total_duration = (frameCount + fps - 1) // fps
            start_sec, end_sec = 0, total_duration
            interval = fps / frame_rate
            frames_idx = np.floor(np.arange(start_sec*fps, end_sec*fps, interval))
            ret = True     
            images = np.zeros([len(frames_idx), 3, size, size], dtype=np.float32)

            for i, idx in enumerate(frames_idx):
                cap.set(cv2.CAP_PROP_POS_FRAMES , idx)
                ret, frame = cap.read()    
                if not ret: break
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)             
                last_frame = i
                images[i,:,:,:] = preprocess(size, Image.fromarray(frame).convert("RGB"))

            images = images[:last_frame+1]
        cap.release()
        video_frames = torch.tensor(images)
        
        return video_frames
    
    def get_video_embeddings(self, path):
        self.model = self.model.eval()
        
        video = self.video2image(path).to(self.device)
        visual_output = self.model(video)

        visual_output = visual_output["image_embeds"]
        visual_output = visual_output / visual_output.norm(dim=-1, keepdim=True)
        visual_output = torch.mean(visual_output, dim=0)
        visual_output = visual_output / visual_output.norm(dim=-1, keepdim=True)
        
        return visual_output

In [5]:
clip = CLIPmodel()

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [13]:
%%time
e = clip.get_video_embeddings('/kaggle/input/yappy-videos/fhd.mp4') # testing on cpu

CPU times: user 20.8 s, sys: 1.31 s, total: 22.2 s
Wall time: 7.46 s


In [7]:
%%time
e = clip.get_video_embeddings('/kaggle/input/yappy-videos/fhd.mp4') # testing on gpu

CPU times: user 14.1 s, sys: 429 ms, total: 14.5 s
Wall time: 4.3 s


# Модель для построения CLIP-эмбеддингов текстов

#### CLIP-embedding for text
#### 17ms on GPU, 50ms on CPU

In [14]:
class Text2CLIPModel:
    def __init__(self):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model = CLIPTextModelWithProjection.from_pretrained("Searchium-ai/clip4clip-webvid150k").to(self.device)
        self.tokenizer = CLIPTokenizer.from_pretrained("Searchium-ai/clip4clip-webvid150k")
        
    def get_text_embedding(self, text):
        inputs = self.tokenizer(text=text, return_tensors="pt").to(self.device)
        outputs = self.model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"])
        final_output = outputs[0] / outputs[0].norm(dim=-1, keepdim=True)
        final_output = final_output.cpu().detach().numpy()
        
        return final_output

In [32]:
Text2CLIP = Text2CLIPModel()

In [47]:
%%time
e = Text2CLIP.get_text_embedding('улыбок тебе дед макар') # test on cpu

CPU times: user 97.6 ms, sys: 1.82 ms, total: 99.4 ms
Wall time: 51.2 ms


In [34]:
%%time
e = Text2CLIP.get_text_embedding('улыбок тебе дед макар') # test on gpu

CPU times: user 17.4 ms, sys: 914 µs, total: 18.3 ms
Wall time: 17.3 ms


In [15]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

# Модель для построения эмбеддингов текстов (по ним будем мэтчить "OCR - запрос" и "ASR - запрос"

#### Getting text embeddings
#### 8ms on GPU, 22 ms on CPU

In [16]:
class Text2MiniLM:
    def __init__(self):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2').to(self.device)
        self.tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
        
    def mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output[0]
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    
    def get_lm_embedding(self, text):
        encoded_input = self.tokenizer([text], padding=True, truncation=True, return_tensors='pt').to(self.device)

        with torch.no_grad():
            model_output = self.model(**encoded_input)

        sentence_embeddings = self.mean_pooling(model_output, encoded_input['attention_mask'])
        sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
        
        return sentence_embeddings

In [41]:
Text2Mini = Text2MiniLM()

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [51]:
%%time
e = Text2Mini.get_lm_embedding('artist drawing some apples') # testing on cpu

CPU times: user 34.6 ms, sys: 2.27 ms, total: 36.8 ms
Wall time: 22.2 ms


In [44]:
%%time
e = Text2Mini.get_lm_embedding('artist drawing some apples') # testing on gpu

CPU times: user 7.32 ms, sys: 2.05 ms, total: 9.37 ms
Wall time: 8.25 ms


In [17]:
from sentence_transformers import util

# Модель распознавания текста на видео

#### OCR (test on 25 second video)
#### 2.23s on GPU, 21s on CPU

In [18]:
import torch.multiprocessing as mp
from concurrent.futures import ThreadPoolExecutor

if torch.cuda.is_available():
    mp.set_start_method('spawn', force=True) # для gpu

In [19]:
class OCRModel:
    def __init__(self):
        self.flag = torch.cuda.is_available()
        self.reader = easyocr.Reader(['en', 'ru'], gpu=self.flag)
        self.translator = MarianTranslator()

    def process_frame(self, frame):
        frame_resized = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)
        outs = self.reader.readtext(frame_resized)
        outs = [x[1] for x in outs if x[2] >= 0.5]
        outs = ['' if x is None else x for x in outs]
        return ' '.join(outs) if len(outs) > 0 else ''

    def get_text_from_video(self, video_path):
        cap = cv2.VideoCapture(video_path)
        frames = []
        frame_rate = cap.get(cv2.CAP_PROP_FPS)
        frame_interval = int(frame_rate) * 3
        frame_count = 0

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            if frame_count % frame_interval == 0:
                frames.append(frame)
            frame_count += 1
        
        cap.release()

        all_outs = []
        with ThreadPoolExecutor() as executor:
            all_outs = list(executor.map(self.process_frame, frames))

        try:
            fin_text = all_outs[0].capitalize()
            for nt in all_outs[1:]:
                for s in range(min([50, len(nt)]), 1, -1):
                    css = nt[:s]
                    if css in fin_text and s > 3:
                        if len(fin_text) - fin_text.rfind(css) - len(css) < 4:
                            fin_text = fin_text[:fin_text.rfind(css)]
                            break
                fin_text += '. ' + nt.capitalize()
        except Exception as e:
            fin_text = ''

        return self.translator.translate(fin_text)

In [48]:
ocr = OCRModel()

Progress: |██████████████████████████████████████████████████| 100.1% Complete

/opt/conda/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [140]:
%%time
ocr.get_text_from_video('/kaggle/input/yappy-videos/fhd (1).mp4') # test on cpu

CPU times: user 1min 5s, sys: 5.64 s, total: 1min 11s
Wall time: 21 s


"Kak's rule is natural, that's the chapter of this tree together with his family going to the region and settled in a row with all the injured ones, they live like this before they can figure it out and now remember the lava of their region"

In [50]:
%%time
ocr.get_text_from_video('/kaggle/input/yappy-videos/fhd (1).mp4') # test on gpu

CPU times: user 5.64 s, sys: 196 ms, total: 5.84 s
Wall time: 2.23 s


"Kak's rule is natural, that's the chapter of this tree together with his family and is settled in a row with all the injured, they live up to that, until they figure it out, and now remember your region's lava"

# Мета-модель, содержащая ASR, OCR, CLIP

#### Изначально в OCRModel не было параллельной обработки фреймов, а распараллеливание происходило в MetaModel между OCR, Whisper и CLIP, однако оказалось, что если распараллелить OCR, а остальное сделать последовательно, то получится наилучшая скорость (OCR был самым долгим из всех, мог занимать до 35 секунд на длинных видео)

#### Meta (test on 25 sec video)
#### 8s on GPU, 32s on CPU

In [20]:
class MetaModel:
    def __init__(self):
        self.whisper = WhisperModel()
        self.ocr = OCRModel()
        self.clipvision = CLIPmodel()
        self.textembedder = Text2MiniLM()
    
    def get_ocr_embeddings(self, video):
        ocr_text = self.ocr.get_text_from_video(video)[:77]
        ocr_emb = self.textembedder.get_lm_embedding(ocr_text).cpu().flatten().detach().numpy()
        return ocr_emb, len(ocr_text)
    
    def get_whisper_embeddings(self, video):
        whisper_text = self.whisper.process_sample(video)[:77]
        whisper_emb = self.textembedder.get_lm_embedding(whisper_text).cpu().flatten().detach().numpy()
        return whisper_emb, len(whisper_text)
    
    def get_clip_embeddings(self, video):
        clip_emb = self.clipvision.get_video_embeddings(video).cpu().flatten().detach().numpy()
        return clip_emb
    
    def get_embeddings(self, video):
        with ThreadPoolExecutor() as executor:
            future_to_method = {
                executor.submit(self.get_ocr_embeddings, video): 'ocr',
                executor.submit(self.get_whisper_embeddings, video): 'whisper',
                executor.submit(self.get_clip_embeddings, video): 'clip'
            }

            results = {}
            for future in concurrent.futures.as_completed(future_to_method):
                method = future_to_method[future]
                try:
                    result = future.result()
                    results[method] = result
                except Exception as exc:
                    print(f'{method} generated an exception: {exc}')

        clip_emb = results['clip']
        ocr_emb, ocr_len = results['ocr']
        whisper_emb, whisper_len = results['whisper']
        
        return clip_emb, ocr_emb, whisper_emb, whisper_len, ocr_len

In [21]:
meta = MetaModel()

config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/290M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.81k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [166]:
%%time
r = meta.get_embeddings('/kaggle/input/yappy-videos/fhd (1).mp4') # test on cpu

/kaggle/input/yappy-videos/fhd (1).mp4
CPU times: user 1min 51s, sys: 9.1 s, total: 2min
Wall time: 31.9 s


In [73]:
%%time
r = meta.get_embeddings('/kaggle/input/yappy-videos/fhd (1).mp4') # test on gpu

/kaggle/input/yappy-videos/fhd (1).mp4
CPU times: user 28 s, sys: 1.05 s, total: 29 s
Wall time: 8.36 s


In [22]:
! pip install clickhouse_connect

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 966.2/966.2 kB 10.9 MB/s eta 0:00:0000:010:01


# Функции для работы с БД Clickhouse

In [23]:
from urllib.request import urlopen
from shutil import copyfileobj
import clickhouse_connect
import tqdm

client = clickhouse_connect.get_client(host='91.224.86.248', port=8123)
TABLENAME = 'embeddings'

def load_from_csv(path, start, count):
    '''
    Вернет список из списков [id, url, text]
    '''
    with open(path, encoding='utf-8') as f:
        text = f.read().split('\nhttps://cdn-st.rutubelist.ru')
    try:
        text = text[start + 1:start + count + 1]
    except IndexError:
        text = text[start + 1:]
    videos = list(map(lambda x: [start + x[0]] + x[1].split(','), enumerate(text)))
    return [[i[0], 'https://cdn-st.rutubelist.ru' + i[1], ','. join(i[2:])] for i in videos]


def download_to(url, path):
    with urlopen(url) as in_stream, open(str(path), 'wb') as out_file:
        copyfileobj(in_stream, out_file)

        
def iteration(fp, count, max_id = -1):
    if max_id == -1:
        max_id = client.query(f'SELECT max(id) FROM {TABLENAME}').result_columns
        print('maxid', max_id)
        max_id = max_id[0][0] + 1
    videos = load_from_csv(fp, max_id, count)
    data = []
    for video in tqdm.tqdm(videos):
        fname = str(video[0]) + '.mp4'
        url = video[1]
        download_to(url, fname)
        print(fname)
        embs = meta.get_embeddings(fname)
        data.append([video[0]] + list(embs))
        os.remove(fname)

    client.insert(TABLENAME, data, column_names=['id', 'clip_emb', 'ocr_emb', 'whisper_emb', 'whisper_len', 'ocr_len'])

In [ ]:
'''НЕКСТ С 56000'''

In [24]:
import clickhouse_connect

client = clickhouse_connect.get_client(host='91.224.86.248', 
                                       port=8123,
                                       username=..., 
                                       password=...)
TABLENAME = 'embeddings'

In [25]:
data = client.query_df(f'SELECT id, clip_emb, ocr_emb, whisper_emb, whisper_len, ocr_len FROM {TABLENAME}')

In [26]:
data = data.drop_duplicates(subset='id')

In [27]:
len(data)

50980

In [ ]:
import concurrent

In [ ]:
# start_id = 36000
# for i in range(100):
#     iteration(r'/kaggle/input/yappy-csv-data/yappy_hackaton_2024_400k.csv', 10, start_id + (i * 10))

In [28]:
df = pd.read_csv('/kaggle/input/yappy-csv-data/yappy_hackaton_2024_400k.csv')

In [29]:
# ! pip install faiss-cpu
! pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 50.2 MB/s eta 0:00:00:00:0100:01


# Класс для ранжирования видео по текстовому запросу

#### Скорость поисковой выдачи (тест при k=20)
#### На GPU 211ms, на CPU 323ms (включая перевод, построение эмбеддингов и сам поиск top-k видео)

In [30]:
import faiss

class SimilarityRanker:
    def __init__(self, db, df):
        self.cliptext = Text2CLIPModel()
        self.textembedder = Text2MiniLM()
        self.translator = MarianTranslator()
        self.db = db
        self.df = df
        self.index_clip = self.create_faiss_index(np.vstack(self.db['clip_emb'].values).astype('float32'))
        self.index_ocr = self.create_faiss_index(np.vstack(self.db['ocr_emb'].values).astype('float32'))
        self.index_whisper = self.create_faiss_index(np.vstack(self.db['whisper_emb'].values).astype('float32'))
        
    def create_faiss_index(self, embeddings):
        dim = embeddings.shape[1]
        index = faiss.IndexFlatL2(dim)
        index.add(embeddings)
        return index
    
    def get_weights(self, word_whisper_count, word_ocr_count):
        max_count = 77
        
        w_1 = (word_ocr_count / (max_count + word_ocr_count))
        w_2 = (word_whisper_count / (max_count + word_whisper_count))
        
        ocr_weight = 0.05 * w_1
        whisper_weight = 0.05 * w_2
        clip_weight = 0.9 * (1 - (w_1 + w_2) / 2)
        
        ocr_weight, whisper_weight, clip_weight = np.exp([ocr_weight, whisper_weight, clip_weight]) / np.sum(np.exp([ocr_weight, whisper_weight, clip_weight]))

        return clip_weight, ocr_weight, whisper_weight
    
    def find_top_k(self, text, k=5):
        text = self.translator.translate(text)
        
        text_emb = self.textembedder.get_lm_embedding(text).cpu().flatten().detach().numpy()
        clip_emb = self.cliptext.get_text_embedding(text).astype('float32')
        text_emb = text_emb.astype('float32')

        _, clip_indices = self.index_clip.search(clip_emb.reshape(1, -1), k)
        _, ocr_indices = self.index_ocr.search(text_emb.reshape(1, -1), k)
        _, whisper_indices = self.index_whisper.search(text_emb.reshape(1, -1), k)
        
        unique_indices = set(clip_indices[0]).union(ocr_indices[0]).union(whisper_indices[0])
        
        best_videos = []
        for i in unique_indices:
            id_, video_clip_emb, video_ocr_emb, video_whisper_emb, word_whisper_count, word_ocr_count = self.db.iloc[i]
            
            video_clip_emb = np.array(video_clip_emb).astype('float32')
            video_ocr_emb = np.array(video_ocr_emb).astype('float32')
            video_whisper_emb = np.array(video_whisper_emb).astype('float32')

            score_clip = util.cos_sim(video_clip_emb.reshape(1, -1), clip_emb.reshape(1, -1))[0][0]
            score_ocr = util.cos_sim(video_ocr_emb.reshape(1, -1), text_emb.reshape(1, -1))[0][0]
            score_whisper = util.cos_sim(video_whisper_emb.reshape(1, -1), text_emb.reshape(1, -1))[0][0]

            clip_weight, ocr_weight, whisper_weight = self.get_weights(word_whisper_count, word_ocr_count)

            total_score = score_clip * clip_weight + score_ocr * ocr_weight + score_whisper * whisper_weight
            
            best_videos.append((total_score, id_))
        
        best_videos.sort(reverse=True, key=lambda x: x[0])
        
        res = {x[0].item(): self.df.iloc[x[1]].link for x in best_videos[:k]}
        
        return res

In [31]:
ranker = SimilarityRanker(data, df)

/opt/conda/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [42]:
%%time
res = ranker.find_top_k('роблокс', k=20) # test on cpu

CPU times: user 587 ms, sys: 1.02 ms, total: 588 ms
Wall time: 323 ms


In [85]:
%%time
res = ranker.find_top_k('роблокс', k=20) # test on gpu

CPU times: user 221 ms, sys: 3.95 ms, total: 225 ms
Wall time: 211 ms


In [32]:
res # можете посмотреть результаты по запросу 'роблокс'

{0.5339266061782837: 'https://cdn-st.rutubelist.ru/media/a1/21/9a16c69e49b9817bd76a0877c044/fhd.mp4',
 0.498274564743042: 'https://cdn-st.rutubelist.ru/media/9d/2f/b055109743e0a0889096a3d7b9fa/fhd.mp4',
 0.4717157483100891: 'https://cdn-st.rutubelist.ru/media/77/4d/6b13f7164a99824f4dbc1d32cb5f/fhd.mp4',
 0.46740278601646423: 'https://cdn-st.rutubelist.ru/media/08/f8/246c3d904de58622247f84e06004/fhd.mp4',
 0.45856237411499023: 'https://cdn-st.rutubelist.ru/media/f9/e9/788a07374672b274d95f59f26d39/fhd.mp4',
 0.4531460702419281: 'https://cdn-st.rutubelist.ru/media/32/22/cf1a07b5469cab6ba8e28befae68/fhd.mp4',
 0.4500095546245575: 'https://cdn-st.rutubelist.ru/media/30/a5/89c7e6a441d893c8d653520410d0/fhd.mp4',
 0.44426268339157104: 'https://cdn-st.rutubelist.ru/media/73/fe/07b47a184ef195521c191323da8a/fhd.mp4',
 0.441667377948761: 'https://cdn-st.rutubelist.ru/media/71/93/435082ad4b67a19408f6cb71f6c5/fhd.mp4',
 0.43777334690093994: 'https://cdn-st.rutubelist.ru/media/eb/ee/367db5d34212a6913

In [33]:
! pip install hdbscan

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 14.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 62.2 MB/s eta 0:00:00
  Attempting uninstall: cython
    Found existing installation: Cython 3.0.8
    Uninstalling Cython-3.0.8:
      Successfully uninstalled Cython-3.0.8


# Кластеризация

#### На этот момент было ~35000 видео, кластеры были рассчитаны именно для такого кол-ва видео

In [ ]:
import pandas as pd
import numpy as np
import hdbscan
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

new_df = data.copy()

embeddings = np.vstack(new_df['clip_emb'].values)

scaler = StandardScaler()
embeddings_scaled = scaler.fit_transform(embeddings)

pca = PCA(n_components=50)
embeddings_reduced = pca.fit_transform(embeddings_scaled)

clusterer = hdbscan.HDBSCAN(min_cluster_size=10, core_dist_n_jobs=-1)
labels = clusterer.fit_predict(embeddings_reduced)

new_df['cluster'] = labels

In [82]:
cluster_dict = new_df.groupby('cluster')['id'].apply(list).to_dict()

In [83]:
for cluster, ids in cluster_dict.items():
    cluster_dict[cluster] = [df.iloc[id_].link for id_ in ids]

In [184]:
cluster_dict[98]

['https://cdn-st.rutubelist.ru/media/a4/b5/4fdd0b2f4bbbbb27890a4c549c8a/fhd.mp4',
 'https://cdn-st.rutubelist.ru/media/6a/ac/815beef14fbcb2e2e7dff3cbad80/fhd.mp4',
 'https://cdn-st.rutubelist.ru/media/87/69/36a18be742a19fadd7471bdaf8b7/fhd.mp4',
 'https://cdn-st.rutubelist.ru/media/19/f8/c92a918645158bd0e3c200488b37/fhd.mp4',
 'https://cdn-st.rutubelist.ru/media/40/1e/0146c060433f87bf4129c26cd302/fhd.mp4',
 'https://cdn-st.rutubelist.ru/media/24/e5/8c97cd9a4c1498c3ba2bee641595/fhd.mp4',
 'https://cdn-st.rutubelist.ru/media/92/65/99cdbd9a477192a23413d59ff80a/fhd.mp4',
 'https://cdn-st.rutubelist.ru/media/0a/dd/53c29ad6492da251ad60941826ae/fhd.mp4',
 'https://cdn-st.rutubelist.ru/media/11/ad/45714d0b479e98f7c38c8b822444/fhd.mp4',
 'https://cdn-st.rutubelist.ru/media/94/9a/e185cbcf4fc98b0f52d42473eb8e/fhd.mp4',
 'https://cdn-st.rutubelist.ru/media/1f/50/a1fd56194a4886db097706867680/fhd.mp4']

## Отсмотрел видосы получившихся кластеров и дал названия...

In [241]:
mapping = {
    #0: 'Раскраски',
    #1: 'Узбекистан',
    2: 'Озвучка', # ЧЕЛОВЕК норм кластер
    3: 'Гитара', # крутой кластер
    #4: 'Странные анимации', 
    #5: 'Автомобили', # ЧЕЛОВЕК
    6: 'ОГЭ И ЕГЭ', # крутой кластер
    7: 'Здоровье', # норм кластер
    8: 'Пародии', # огромный кластер со странным контентом
    #9: 'ТЦ Пранки?', # ЧЕЛОВЕК
    10: 'Математика', # ЧЕЛОВЕК крутой кластер
    11: 'Математика', # 2 человека крутой кластер
    #12: 'Музыка 80-х и 90-х', # норм кластер
    #13: 'Сгенерировала нейросеть', # норм кластер
    14: 'FIFA', # норм кластер
    15: 'Хоккей', # крутой кластер
    16: 'Фигурное катание', # крутой кластер
    #17: 'Перемешались 2 чела и дота', # дропнуть
    #18: 'Геймдев', # ЧЕЛОВЕК
    19: 'Мотоциклы', # норм кластер
    20: 'Причёски', # крутой кластер 
    #21: 'Таро', # до связи...
    #22: 'Дикая природа', # дроп?
    #23: 'Снежная королева', # дроп?
    24: 'Карточные игры', # норм кластер
    #25: 'Всратые скетчи', # ЧЕЛОВЕК
    26: 'Единоборства', # норм кластер
    27: 'Футбол', # крутой кластер
    28: 'Баскетбол', # крутой кластер
    #29: 'Гадалка со странными видео', # ЧЕЛОВЕК
    30: 'Мотоциклы', # норм кластер, объединить с 19 тут в лесу и еще в странных местах
    31: 'Мотоциклы', # норм кластер, объединить с 19 тут в городе
    32: 'Мотоциклы', # норм кластер, объединить с 19 тут скорее гонки
    #33: 'Бля))', # дроп
    #34: 'Массаж', # хз наверное дроп
    35: 'Животные', # норм кластер точно оставить
    #36: 'Прыжки с парашюта', # хз мб норм
    #37: 'Ааа женщина', # дроп
    #38: 'Бубс', # дроп
    #39: 'Почти бубс', # дроп
    #40: 'Японские девочки...',
    #41: 'Опять бубс', # дроп
    #42: 'Видео про футбол', # ЧЕЛОВЕК
    #43: 'Мужик в трусах', # дроп
    #44: 'Небоскребы', # хз вроде и норм но зачем 
    45: 'Одежда', 
    46: 'Мода',
    47: 'Аутфиты', # ОГРОМНЫЙ кластер надо оставить
    48: 'Клавиатуры', # кайф кластер оставляем
    #49: 'Растяжка', # дроп
    50: 'Тренировки', # нуу норм
    51: 'Здоровье', # ЧЕЛОВЕК
    52: 'Фитнес', # ну это еще норм
    53: 'Красоты городов', # нормик
    54: 'Сноубординг', # крутой кластер
    55: 'Футбол', # склеить с 27
    #56: 'Какой-то мульт я хз', # дроп
    57: 'Мультфильмы', # норм кластер
    58: 'Единоборства', # склеить с 26
    59: 'Природа', # крутой кластер
    60: 'Горы', # норм кластер мб склеить с 59
    61: 'Майнкрафт', # нуу вроде норм
    62: 'Сёрфинг', # норм кластер
    63: 'Море', # крутой кластер
    64: 'Рецепты', # хороший кластер
    65: 'DIY & Art', # странный кластер
    #66: 'Странная игра', # дроп
    #67: 'GTA', # ЧЕЛОВЕК +немного мусора
    68: 'Уход за собой', # мб норм
    69: 'Dota 2', # кайф кластер
    70: 'Маникюр', # мб норм
    71: 'Макияж', # ОГРОМНЫЙ кластер оставляем
    72: 'Brawl Stars', # я его нашел
    73: 'Игровые приставки и ПК', # нормик
    74: 'Смартфоны и гаджеты', # большой кластер норм
    #75: 'Чел залил миллион однотипных видосов в подписями', # ЧЕЛОВЕК
    #76: 'Коллекционные автомобили', # норм но хз зачем
    77: 'Смешарики', # кайф кластер
    #78: 'Опять скетчи', # ЧЕЛОВЕК
    #79: 'Всякие истории', # ЧЕЛОВЕК
    80: 'Кинематограф', # ЧЕЛОВЕК норм кластер
    #81: 'Counter-Strike', # ЧЕЛОВЕК
    #82: 'Игры в которые играет твой батя', # хз странно
    83: 'Автомобили', # большой кластер пойдет
    #84: 'Зиро коментс', # хд
    #85: 'Намешано все', # дроп
    86: 'Истории о мультфильмах', # ЧЕЛОВЕК +- норм
    #87: 'Новости Казахстана', # втф...
    #88: 'Алкоголизм', # ЧЕЛОВЕК мужик ты куда
    #89: 'Отношения', # ЧЕЛОВЕК хз дроп думаю 
    90: 'Интересные факты', # реально норм вроде
    91: 'Фрагменты из фильмов', # хз +- норм
    92: 'Аниме', # ОГРОМНЫЙ кластер оставляем
    93: 'Genshin Impact', # ну почему бы и нет))
    94: 'Мультфильмы', # приклеить к 57
    #95: 'Смешались рассказы про аниме и геншин', # тут мало можно дропать
    #96: 'Опять таро..', # дроп
    97: 'Исторические факты', # хороший кластер
    98: 'Фрагменты из фильмов', # склеить с 91
}

In [262]:
to_drop = [-1, 0, 1, 4, 5, 9, 12, 13, 17, 18, 21, 22, 
           23, 25, 29, 33, 34, 36, 37, 38, 39, 40, 
           41, 42, 43, 44, 49, 56, 66, 67, 75, 76, 
           78, 79, 81, 82, 84, 85, 87, 88, 89, 95, 96]

In [282]:
df_clusters = pd.DataFrame(new_df.groupby('cluster')['id'].apply(list))

In [283]:
df_clusters['links'] = df_clusters['id'].apply(lambda ids: [df.iloc[id_].link for id_ in ids]) # получим ссылки по айди

In [284]:
df_clusters = df_clusters.reset_index()

In [295]:
df_clusters = df_clusters[~df_clusters['cluster'].isin(to_drop)] # дропнем малочисленные и странные кластеры

In [297]:
df_clusters['name'] = [mapping[id_] for id_ in df_clusters.cluster] # дадим кластерам имена

/tmp/ipykernel_34/499800031.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clusters['name'] = [mapping[id_] for id_ in df_clusters.cluster]


In [304]:
def combine_lists(x):
    combined = []
    for sublist in x:
        combined.extend(sublist)
    return list(set(combined))


agg_func = {
    'cluster': 'first',
    'id': combine_lists,
    'links': combine_lists
}

df_clusters_new = df_clusters.groupby('name').agg(agg_func).reset_index() # повторяющиеся имена объединим в одинаковые кластеры

In [305]:
df_clusters_new

,name,cluster,id,links
0,Brawl Stars,72,"[36736, 20354, 34818, 20998, 28039, 27784, 321...",[https://cdn-st.rutubelist.ru/media/88/e4/0f10...
1,DIY & Art,65,"[4609, 12292, 30214, 28168, 14344, 12520, 2254...",[https://cdn-st.rutubelist.ru/media/37/5a/6696...
2,Dota 2,69,"[22533, 30215, 18952, 28171, 28688, 5649, 8210...",[https://cdn-st.rutubelist.ru/media/ab/9c/e75e...
3,FIFA,14,"[7936, 6918, 25992, 23050, 31501, 36365, 25746...",[https://cdn-st.rutubelist.ru/media/51/df/540f...
4,Genshin Impact,93,"[15872, 19144, 31627, 21711, 25488, 10223, 500...",[https://cdn-st.rutubelist.ru/media/49/1d/de42...
5,Автомобили,83,"[2048, 22021, 3080, 19978, 26122, 32267, 2573,...",[https://cdn-st.rutubelist.ru/media/47/27/b97d...
6,Аниме,92,"[16397, 30734, 2062, 15, 22545, 4115, 35, 2256...",[https://cdn-st.rutubelist.ru/media/98/a6/96aa...
7,Аутфиты,47,"[18437, 22535, 14345, 10292, 4149, 6203, 12348...",[https://cdn-st.rutubelist.ru/media/67/d1/1d9f...
8,Баскетбол,28,"[18820, 22149, 9227, 530, 3475, 36370, 17298, ...",[https://cdn-st.rutubelist.ru/media/ed/c7/abfd...
9,Гитара,3,"[33416, 7436, 25978, 1682, 26516, 30359, 7704,...",[https://cdn-st.rutubelist.ru/media/f3/03/7bdc...


In [306]:
df_clusters_new.to_csv('final_clusters.csv') # сохраним полученные кластеры